In [1]:
import getpass, pwd, os
def thing():
    print(os.getuid())
    print(open("/etc/passwd", 'r').readlines())
    print(pwd.getpwnam('ubuntu'))
    print(pwd.getpwuid(12574))
    print(pwd.getpwuid(os.getuid())[0])
    print(getpass.getuser())

In [2]:
thing()

12574
['root:x:0:0:root:/root:/bin/bash\n', 'daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin\n', 'bin:x:2:2:bin:/bin:/usr/sbin/nologin\n', 'sys:x:3:3:sys:/dev:/usr/sbin/nologin\n', 'sync:x:4:65534:sync:/bin:/bin/sync\n', 'games:x:5:60:games:/usr/games:/usr/sbin/nologin\n', 'man:x:6:12:man:/var/cache/man:/usr/sbin/nologin\n', 'lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin\n', 'mail:x:8:8:mail:/var/mail:/usr/sbin/nologin\n', 'news:x:9:9:news:/var/spool/news:/usr/sbin/nologin\n', 'uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin\n', 'proxy:x:13:13:proxy:/bin:/usr/sbin/nologin\n', 'www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin\n', 'backup:x:34:34:backup:/var/backups:/usr/sbin/nologin\n', 'list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin\n', 'irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin\n', 'gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin\n', 'nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin\n', '_apt:x:100:65534::/nonexi

In [3]:
import ray
import xgboost_ray as xgbr

if ray.is_initialized() == False:
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    ray.init(f"ray://{service_host}:{service_port}")

In [6]:
ray.nodes()

[{'NodeID': 'd97394a6ea539b57f821060faab976782422d6c6c3560bee2ca5ac65',
  'Alive': True,
  'NodeManagerAddress': '10.0.123.157',
  'NodeManagerHostname': 'ray-6351a2d3c533470d0466edaa-ray-worker-0',
  'NodeManagerPort': 2385,
  'ObjectManagerPort': 2384,
  'ObjectStoreSocketName': '/tmp/ray/session_2022-10-20_12-34-48_107209_1/sockets/plasma_store',
  'RayletSocketName': '/tmp/ray/session_2022-10-20_12-34-48_107209_1/sockets/raylet',
  'MetricsExportPort': 61123,
  'alive': True,
  'Resources': {'object_store_memory': 1267046400.0,
   'memory': 2956441600.0,
   'node:10.0.123.157': 1.0,
   'CPU': 1.0}},
 {'NodeID': 'd4b67c6b643b6cb347d7b0503d7d5bee84ca1e3f23a3d2911f6ed67b',
  'Alive': True,
  'NodeManagerAddress': '10.0.33.157',
  'NodeManagerHostname': 'ray-6351a2d3c533470d0466edaa-ray-worker-1',
  'NodeManagerPort': 2385,
  'ObjectManagerPort': 2384,
  'ObjectStoreSocketName': '/tmp/ray/session_2022-10-20_12-34-48_107209_1/sockets/plasma_store',
  'RayletSocketName': '/tmp/ray/sessio

In [4]:
@ray.remote
def tryit():
    thing()

In [5]:
ray.get(tryit.remote())

(tryit pid=2267) 12574
(tryit pid=2267) ['root:x:0:0:root:/root:/bin/bash\n', 'daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin\n', 'bin:x:2:2:bin:/bin:/usr/sbin/nologin\n', 'sys:x:3:3:sys:/dev:/usr/sbin/nologin\n', 'sync:x:4:65534:sync:/bin:/bin/sync\n', 'games:x:5:60:games:/usr/games:/usr/sbin/nologin\n', 'man:x:6:12:man:/var/cache/man:/usr/sbin/nologin\n', 'lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin\n', 'mail:x:8:8:mail:/var/mail:/usr/sbin/nologin\n', 'news:x:9:9:news:/var/spool/news:/usr/sbin/nologin\n', 'uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin\n', 'proxy:x:13:13:proxy:/bin:/usr/sbin/nologin\n', 'www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin\n', 'backup:x:34:34:backup:/var/backups:/usr/sbin/nologin\n', 'list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin\n', 'irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin\n', 'gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin\n', 'nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nolo

In [6]:
from sklearn.datasets import fetch_california_housing
ca_data = fetch_california_housing(as_frame=True)
ca_data.frame.to_csv("cadata.csv", index=False)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_poisson_deviance
import pandas as pd
import xgboost as xgb

In [8]:
all_data = pd.read_csv("cadata.csv")
# Probably going to save separate files for train/test data for the ray examples
#  so I can just bypass the many other options for splitting
train_data, test_data = train_test_split(all_data, test_size=0.3, random_state=42)

In [9]:
target_col = "MedHouseVal"
X_train = train_data.drop(target_col, axis=1)
y_train = train_data[target_col]
X_test = test_data.drop(target_col, axis=1)
y_test = test_data[target_col]

In [10]:
dtrain = xgbr.RayDMatrix(X_train, y_train)
dtest = xgbr.RayDMatrix(X_test, y_test)

In [11]:
tune_results_dir = os.path.join("/domino/datasets/local", os.environ["DOMINO_PROJECT_NAME"], "ray_results")
print(f"Saving intermediate tune results to {tune_results_dir}")

Saving intermediate tune results to /domino/datasets/local/Ray-Tutorial/ray_results


In [12]:
from ray import tune

In [13]:
ray_param = xgbr.RayParams(num_actors=2, cpus_per_actor=1)
ray_param.get_tune_resources().required_resources

{'CPU': 3.0}

In [14]:
# Reference the xgboost parameters link again here!

def my_trainer(config):
    evals_result = {}
    bst = xgbr.train(
        params=config,
        dtrain=dtrain, # This is some spooky global variable stuff, reusing my dtrain from previous sections
        evals_result=evals_result,
        evals=[(dtrain, "train")], # oohhh is this necessary to get "train-rmse" coming out correctly??
        ray_params=ray_param # ok this seems wrong to me, why give the same resources here and to Tune?
    )
    # xgbr.train claims it already knows how to report back to Ray so I guess this is it?
    #bst.save_model("model.xgb")

config = {
    "tree_method": "approx",
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "eta": tune.loguniform(1e-4, 1e-1),
    "max_depth": tune.randint(1, 9)
}

analysis = tune.run(
    my_trainer,
    config=config,
    local_dir=tune_results_dir, # gotta change this to avoid permission denied on ~/ray_results
    sync_config=tune.SyncConfig(syncer=None), # gotta change this to avoid weird "getpwuid(): uid not found: 12574"
    metric="train-rmse",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_param.get_tune_resources()
)

(run pid=218) 2022-10-16 13:00:55,826	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


(run pid=218) == Status ==
(run pid=218) Current time: 2022-10-16 13:00:56 (running for 00:00:00.21)
(run pid=218) Memory usage on this node: 5.0/30.9 GiB
(run pid=218) Using FIFO scheduling algorithm.
(run pid=218) Resources requested: 0/3 CPUs, 0/0 GPUs, 0.0/7.86 GiB heap, 0.0/3.54 GiB objects
(run pid=218) Result logdir: /domino/datasets/local/Ray-Tutorial/ray_results/my_trainer_2022-10-16_13-00-55
(run pid=218) Number of trials: 4/4 (4 PENDING)
(run pid=218) +------------------------+----------+-------+-------------+-------------+
(run pid=218) | Trial name             | status   | loc   |         eta |   max_depth |
(run pid=218) |------------------------+----------+-------+-------------+-------------|
(run pid=218) | my_trainer_3eda2_00000 | PENDING  |       | 0.000673909 |           8 |
(run pid=218) | my_trainer_3eda2_00001 | PENDING  |       | 0.0236393   |           3 |
(run pid=218) | my_trainer_3eda2_00002 | PENDING  |       | 0.088005    |           8 |
(run pid=218) | my_

(ImplicitFunc pid=77, ip=10.0.50.190) 2022-10-16 13:01:00,726	INFO main.py:926 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.


(run pid=218) == Status ==
(run pid=218) Current time: 2022-10-16 13:01:01 (running for 00:00:05.26)
(run pid=218) Memory usage on this node: 5.4/30.9 GiB
(run pid=218) Using FIFO scheduling algorithm.
(run pid=218) Resources requested: 3.0/3 CPUs, 0/0 GPUs, 0.0/7.86 GiB heap, 0.0/3.54 GiB objects
(run pid=218) Result logdir: /domino/datasets/local/Ray-Tutorial/ray_results/my_trainer_2022-10-16_13-00-55
(run pid=218) Number of trials: 4/4 (3 PENDING, 1 RUNNING)
(run pid=218) +------------------------+----------+----------------+-------------+-------------+
(run pid=218) | Trial name             | status   | loc            |         eta |   max_depth |
(run pid=218) |------------------------+----------+----------------+-------------+-------------|
(run pid=218) | my_trainer_3eda2_00000 | RUNNING  | 10.0.50.190:77 | 0.000673909 |           8 |
(run pid=218) | my_trainer_3eda2_00001 | PENDING  |                | 0.0236393   |           3 |
(run pid=218) | my_trainer_3eda2_00002 | PENDING 

(_RemoteRayXGBoostActor pid=348) [13:01:04] task [xgboost.ray]:140234718513616 got new rank 0
(ImplicitFunc pid=77, ip=10.0.50.190) 2022-10-16 13:01:04,646	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=104, ip=10.0.92.121) [13:01:04] task [xgboost.ray]:140281395173840 got new rank 1


...

(run pid=218) == Status ==
(run pid=218) Current time: 2022-10-16 13:01:20 (running for 00:00:23.36)
(run pid=218) Memory usage on this node: 5.2/30.9 GiB
(run pid=218) Using FIFO scheduling algorithm.
(run pid=218) Resources requested: 3.0/3 CPUs, 0/0 GPUs, 0.0/7.86 GiB heap, 0.0/3.54 GiB objects
(run pid=218) Current best trial: 3eda2_00002 with train-rmse=0.924104 and parameters={'tree_method': 'approx', 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.08800498336835186, 'max_depth': 8, 'nthread': 1, 'n_jobs': 1}
(run pid=218) Result logdir: /domino/datasets/local/Ray-Tutorial/ray_results/my_trainer_2022-10-16_13-00-55
(run pid=218) Number of trials: 4/4 (1 RUNNING, 3 TERMINATED)
(run pid=218) +------------------------+------------+-----------------+-------------+-------------+--------+------------------+--------------+
(run pid=218) | Trial name             | status     | loc             |         eta |   max_depth |   iter |   total time (s) |   train-rmse |
(run p

(ImplicitFunc pid=278, ip=10.0.50.190) 2022-10-16 13:01:20,088	INFO main.py:926 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=278, ip=10.0.50.190) 2022-10-16 13:01:23,340	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=248, ip=10.0.92.121) [13:01:23] task [xgboost.ray]:140193012869584 got new rank 1
(_RemoteRayXGBoostActor pid=662) [13:01:23] task [xgboost.ray]:140106590276048 got new rank 0


(run pid=218) Result for my_trainer_3eda2_00003:
(run pid=218)   date: 2022-10-16_13-01-24
(run pid=218)   done: false
(run pid=218)   experiment_id: 6f2ac78fc4784f9d8552c0c185ab4f96
(run pid=218)   hostname: ray-634c6195c533470d0466a970-ray-worker-0
(run pid=218)   iterations_since_restore: 1
(run pid=218)   node_ip: 10.0.50.190
(run pid=218)   pid: 278
(run pid=218)   time_since_restore: 4.30981707572937
(run pid=218)   time_this_iter_s: 4.30981707572937
(run pid=218)   time_total_s: 4.30981707572937
(run pid=218)   timestamp: 1665950484
(run pid=218)   timesteps_since_restore: 0
(run pid=218)   train-rmse: 1.914862
(run pid=218)   training_iteration: 1
(run pid=218)   trial_id: 3eda2_00003
(run pid=218)   
(run pid=218) == Status ==
(run pid=218) Current time: 2022-10-16 13:01:24 (running for 00:00:27.71)
(run pid=218) Memory usage on this node: 5.6/30.9 GiB
(run pid=218) Using FIFO scheduling algorithm.
(run pid=218) Resources requested: 3.0/3 CPUs, 0/0 GPUs, 0.0/7.86 GiB heap, 0.0

(ImplicitFunc pid=278, ip=10.0.50.190) 2022-10-16 13:01:24,545	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=14,448 in 4.49 seconds (1.20 pure XGBoost training time).
(run pid=218) 2022-10-16 13:01:24,706	INFO tune.py:626 -- Total run time: 28.89 seconds (27.88 seconds for the tuning loop).


(run pid=218) == Status ==
(run pid=218) Current time: 2022-10-16 13:01:24 (running for 00:00:27.91)
(run pid=218) Memory usage on this node: 5.2/30.9 GiB
(run pid=218) Using FIFO scheduling algorithm.
(run pid=218) Resources requested: 0/3 CPUs, 0/0 GPUs, 0.0/7.86 GiB heap, 0.0/3.54 GiB objects
(run pid=218) Current best trial: 3eda2_00002 with train-rmse=0.924104 and parameters={'tree_method': 'approx', 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.08800498336835186, 'max_depth': 8, 'nthread': 1, 'n_jobs': 1}
(run pid=218) Result logdir: /domino/datasets/local/Ray-Tutorial/ray_results/my_trainer_2022-10-16_13-00-55
(run pid=218) Number of trials: 4/4 (4 TERMINATED)
(run pid=218) +------------------------+------------+-----------------+-------------+-------------+--------+------------------+--------------+
(run pid=218) | Trial name             | status     | loc             |         eta |   max_depth |   iter |   total time (s) |   train-rmse |
(run pid=218) |----

In [17]:
analysis.best_config

{'tree_method': 'approx',
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'eta': 0.08800498336835186,
 'max_depth': 8}

In [18]:
analysis.best_result

{'train-rmse': 0.924104,
 'time_this_iter_s': 0.007105350494384766,
 'done': True,
 'timesteps_total': None,
 'episodes_total': None,
 'training_iteration': 10,
 'trial_id': '3eda2_00002',
 'experiment_id': 'c1614493f6114fc69afc8a1c8f94c831',
 'date': '2022-10-16_13-01-18',
 'timestamp': 1665950478,
 'time_total_s': 3.951627254486084,
 'pid': 183,
 'hostname': 'ray-634c6195c533470d0466a970-ray-worker-0',
 'node_ip': '10.0.50.190',
 'config': {'tree_method': 'approx',
  'objective': 'reg:squarederror',
  'eval_metric': 'rmse',
  'eta': 0.08800498336835186,
  'max_depth': 8,
  'nthread': 1,
  'n_jobs': 1},
 'time_since_restore': 3.951627254486084,
 'timesteps_since_restore': 0,
 'iterations_since_restore': 10,
 'experiment_tag': '2_eta=0.088005,max_depth=8'}